## Example of querying the Ollama Inference server with Mistral-7b, Langchain and a custom Prompt

### Set the Inference server url (replace with your own address)

In [1]:
# Replace values according to your Ollama deployment
MODEL = "mistral:latest"
# BASE_URL = "https://ollama-route-<your-namespace>.apps.shift.nerc.mghpcc.org"
BASE_URL = "http://ollama-service:11434"
# BASE_URL = "http://ollama-service.<your-namespace>.svc.cluster.local:11434"

## Preparation

In [2]:
!pip install --upgrade pip
!pip install --upgrade langchain_openai qdrant_client langchain_qdrant langchain_community
!pip install --upgrade langchain-ollama
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 239.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 174.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 485.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 382.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 336.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3━━━━━━━━━━━━━━━━━━━━━━━  3/22 [numpy]
  Attempting uninstall: pydantic0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/22 [jiter]
    Found existing installation: pydantic 1.10.14━━━━━━━━━━━━━  4/22 [jiter]
    Uninstalling pydantic-1.10.14:━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/22 [jiter]
      Successfully uninstalled pydantic-1.10.14━━━━━━━━━━━━━━━  4/22 [jiter]
  Attempting uninstall: lan

In [3]:
# Imports
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

#### Create the LLM instance

In [4]:
# LLM definition
llm = Ollama(
    base_url=BASE_URL,
    model=MODEL,
    top_p=0.92,
    temperature=0.01,
    num_predict=512,
    repeat_penalty=1.03,
    callbacks=[StreamingStdOutCallbackHandler()]
)

/tmp/ipykernel_1635/213135327.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


#### Create the Prompt

In [5]:
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
{history}
Human: {input}
AI:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

#### And finally some memory for the conversation

In [6]:
memory=ConversationBufferMemory()

/tmp/ipykernel_1635/2290902444.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()


## First example, fully verbose mode

#### Create the Chain using the different components

In [7]:
# Verbose mode is intentionally set to True so you can see the prompt and the history from the buffer memory
conversation = ConversationChain(llm=llm,
                                 prompt=PROMPT,
                                 verbose=True,
                                 memory=memory
                                )

/tmp/ipykernel_1635/3407751332.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm,


#### Let's talk...

In [8]:
first_input = "Describe Paris in 100 words or less."
conversation.predict(input=first_input);



> Entering new ConversationChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:

Human: Describe Paris in 100 words or less.
AI:
[/INST]

 Paris, the City of Light, is a vibrant metropolis renowned for its rich history, art, and culture. Known as the global center of fashion, gastronomy, and philosophy, it's home to iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Its charming streets, filled with cafes, boutiques, and art 

In [9]:
# This is a follow-up question without context to showcase the conversation memory
second_input = "Is there a river?"
conversation.predict(input=second_input)



> Entering new ConversationChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Current conversation:
Human: Describe Paris in 100 words or less.
AI:  Paris, the City of Light, is a vibrant metropolis renowned for its rich history, art, and culture. Known as the global center of fashion, gastronomy, and philosophy, it's home to iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Its charming streets, filled with cafes, boutiques, and art galleries,

' Yes, the River Seine flows through Paris, adding to its picturesque beauty. The Seine is not only a significant waterway but also a cultural and historical landmark, with many famous sites located along its banks, such as the Louvre Museum'

### Second example, no verbosity

In [10]:
# Verbose mode is intentionally set to True so you can see the prompt and the history from the buffer memory
conversation2 = ConversationChain(llm=llm,
                                 prompt=PROMPT,
                                 verbose=False,
                                 memory=memory
                                )

In [11]:
# Let's clear the previous conversation first
memory.clear()

In [12]:
first_input = "Can you describe London in 100 words?"
conversation2.predict(input=first_input)

 London, the capital city of England and the United Kingdom, is a vibrant, historic, and cosmopolitan metropolis. Known for its iconic landmarks like the Tower of London, Buckingham Palace, and the London Eye, it's a city steeped in rich history. The River Thames, flowing through its heart, offers stunning views of the city's architecture, from modern skyscrapers to ancient buildings. London's cultural scene is diverse and dynamic, with world-class museums, galleries, and theaters. It's a global hub for finance, fashion, arts, and entertainment. Its multicultural population contributes to its rich culinary scene, offering a wide variety of international cuisines. London, a city that never sleeps, is a must-visit destination for travelers seeking a blend of tradition and modernity.

" London, the capital city of England and the United Kingdom, is a vibrant, historic, and cosmopolitan metropolis. Known for its iconic landmarks like the Tower of London, Buckingham Palace, and the London Eye, it's a city steeped in rich history. The River Thames, flowing through its heart, offers stunning views of the city's architecture, from modern skyscrapers to ancient buildings. London's cultural scene is diverse and dynamic, with world-class museums, galleries, and theaters. It's a global hub for finance, fashion, arts, and entertainment. Its multicultural population contributes to its rich culinary scene, offering a wide variety of international cuisines. London, a city that never sleeps, is a must-visit destination for travelers seeking a blend of tradition and modernity."

In [ ]:
# This is a follow-up question without context to showcase the conversation memory
second_input = "Is there a river?"
conversation2.predict(input=second_input)